In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Leer archivo cvs y poner en df
df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")

In [3]:
# Crear una pipeline para el data wrangling
# Se salva el PassengerId que es el índice
index = df['PassengerId']
# Se desglosan en nuevas columnas tanto PassengerId como Cabin
df[['Group', 'NumberGroup']] = df['PassengerId'].str.split('_', expand=True)
df[['Deck', 'Num', "Side"]] = df['Cabin'].str.split('/', expand=True)
# Se suprimen las variables que son redundantes
df = df.drop(['PassengerId','Cabin',"NumberGroup","Name"], axis = 1)
df["Num"] = df["Num"].astype(float).round(0)
df["Group"] = df["Group"].astype(float).round(0)

In [4]:
df.describe(include="all")

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,Deck,Num,Side
count,8492,8476,8511,8514.000000,8490,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693,8693.000000,8494,8494.000000,8494
unique,3,2,3,NaN,2,NaN,NaN,NaN,NaN,NaN,2,NaN,8,NaN,2
top,Earth,False,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,True,NaN,F,NaN,S
freq,4602,5439,5915,NaN,8291,NaN,NaN,NaN,NaN,NaN,4378,NaN,2794,NaN,4288
mean,NaN,NaN,NaN,28.827930,NaN,224.687617,458.077203,173.729169,311.138778,304.854791,NaN,4633.389624,NaN,600.367671,NaN
std,NaN,NaN,NaN,14.489021,NaN,666.717663,1611.489240,604.696458,1136.705535,1145.717189,NaN,2671.028856,NaN,511.867226,NaN
min,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,NaN,0.000000,NaN
25%,NaN,NaN,NaN,19.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2319.000000,NaN,167.250000,NaN
50%,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,4630.000000,NaN,427.000000,NaN
75%,NaN,NaN,NaN,38.000000,NaN,47.000000,76.000000,27.000000,59.000000,46.000000,NaN,6883.000000,NaN,999.000000,NaN


In [5]:
# Se hace la transformacion scaler y one hot encoding
dummies = pd.DataFrame()
for col in df:
    if df[col].dtype == "object":  # Si la columna es de tipo object (categórica)
        # Crear columnas dummy para la columna categórica
        dummy_cols = pd.get_dummies(df[col], prefix=col)
        # Concatenar las columnas dummy al DataFrame dummies
        dummies = pd.concat([dummies, dummy_cols], axis=1)
    else:  # Si la columna no es de tipo object, agregarla tal cual al DataFrame de dummies
        dummies[col] = df[col]

df = dummies

# Se buscan la cantidad de datos ausentes
df.isnull().sum()

HomePlanet_Earth               0
HomePlanet_Europa              0
HomePlanet_Mars                0
CryoSleep_False                0
CryoSleep_True                 0
Destination_55 Cancri e        0
Destination_PSO J318.5-22      0
Destination_TRAPPIST-1e        0
Age                          179
VIP_False                      0
VIP_True                       0
RoomService                  181
FoodCourt                    183
ShoppingMall                 208
Spa                          183
VRDeck                       188
Transported                    0
Group                          0
Deck_A                         0
Deck_B                         0
Deck_C                         0
Deck_D                         0
Deck_E                         0
Deck_F                         0
Deck_G                         0
Deck_T                         0
Num                          199
Side_P                         0
Side_S                         0
dtype: int64

In [6]:
# La idea es imputar los ausentes con un loop con randomforest
# Iterar sobre las columnas del DataFrame
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

df_imputado = df.copy()
max_iter = 5  # Número máximo de vueltas por si queda atascado
i = 0

while df_imputado.isnull().sum().sum() > 0 and i < max_iter:
    print(f"Iteración {i+1}")
    for col in df_imputado.columns:
        if df_imputado[col].isnull().sum() == 0:
            continue

        # Separar filas donde col NO es NaN
        train_data = df_imputado[df_imputado[col].notnull()]
        X_train = train_data.drop(columns=[col])
        y_train = train_data[col]

        # Eliminar filas con NaN en X_train
        train_mask = X_train.notnull().all(axis=1)
        X_train = X_train[train_mask]
        y_train = y_train[train_mask]

        # Separar filas donde col ES NaN
        test_data = df_imputado[df_imputado[col].isnull()]
        X_test = test_data.drop(columns=[col])

        # Eliminar filas con NaN en X_test
        X_test_clean = X_test[X_test.notnull().all(axis=1)]
        valid_indices = X_test_clean.index

        if len(valid_indices) == 0:
            continue

        # Elegir modelo según tipo de dato
        if y_train.dtype == 'object' or y_train.dtype == 'bool':
            model = RandomForestClassifier(n_estimators=100, random_state=0)
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=0)

        # Entrenar y predecir
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test_clean)

        # Conversión si es bool o categórica
        if y_train.dtype == 'bool':
            y_pred = y_pred.astype(bool)
        elif y_train.dtype == 'object':
            y_pred = pd.Series(y_pred).astype(df_imputado[col].dtype).values

        # Imputar
        df_imputado.loc[valid_indices, col] = y_pred

    i += 1

print("Imputación completada.")
# Se eliminan los NAN que hayan quedado
df = df_imputado.dropna()

Iteración 1
Iteración 2
Iteración 3
Iteración 4
Iteración 5
Imputación completada.


In [7]:
# Preparamos conjunto de train
X_train = df.drop(["Transported"], axis=1)
y_train = df["Transported"]

# Entrenamos el modelo clasificador

# Definir el modelo base
rf = RandomForestClassifier(random_state=42)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [200, 300],       # Número de árboles
    'max_depth': [10, 20],      # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],      # Min muestras para dividir nodo
    'min_samples_leaf': [1, 2, 4],        # Min muestras en hoja
    'bootstrap': [True, False]             # Método de muestreo para árboles
}

# Crear el objeto GridSearchCV
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,               # Número de folds para cross-validation
    n_jobs=-1,          # Usar todos los núcleos disponibles
    verbose=1           # Mostrar progreso
)

# Ajustar (entrenar) con los datos
best_model = grid_search.fit(X_train, y_train)

# Resultados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score CV:", grid_search.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores hiperparámetros: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Mejor score CV: 0.783536231884058


In [8]:
# Se carga el archivo de test y se realizan las transformaciones
df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
print(df.shape)
index = df['PassengerId']
# Se desglosan en nuevas columnas tanto PassengerId como Cabin
df[['Group', 'NumberGroup']] = df['PassengerId'].str.split('_', expand=True)
df[['Deck', 'Num', "Side"]] = df['Cabin'].str.split('/', expand=True)
# Se suprimen las variables que son redundantes
df = df.drop(['PassengerId','Cabin',"NumberGroup","Name"], axis = 1)
df["Num"] = df["Num"].astype(float).round(0)
df["Group"] = df["Group"].astype(float).round(0)
dummies = pd.DataFrame()
for col in df:
    if df[col].dtype == "object":  # Si la columna es de tipo object (categórica)
        # Crear columnas dummy para la columna categórica
        dummy_cols = pd.get_dummies(df[col], prefix=col)
        # Concatenar las columnas dummy al DataFrame dummies
        dummies = pd.concat([dummies, dummy_cols], axis=1)
    else:  # Si la columna no es de tipo object, agregarla tal cual al DataFrame de dummies
        dummies[col] = df[col]

df = dummies
df_imputado = df.copy()
# Imputamos a mano la edad
age_mean = df_imputado["Age"].mean()
df_imputado["Age"] = df["Age"].fillna(age_mean)

max_iter = 5  # Número máximo de vueltas por si queda atascado
i = 0

while df_imputado.isnull().sum().sum() > 0 and i < max_iter:
    print(f"Iteración {i+1}")
    for col in df_imputado.columns:
        if df_imputado[col].isnull().sum() == 0:
            continue

        # Separar filas donde col NO es NaN
        train_data = df_imputado[df_imputado[col].notnull()]
        X_train = train_data.drop(columns=[col])
        y_train = train_data[col]

        # Eliminar filas con NaN en X_train
        train_mask = X_train.notnull().all(axis=1)
        X_train = X_train[train_mask]
        y_train = y_train[train_mask]

        # Separar filas donde col ES NaN
        test_data = df_imputado[df_imputado[col].isnull()]
        X_test = test_data.drop(columns=[col])

        # Eliminar filas con NaN en X_test
        X_test_clean = X_test[X_test.notnull().all(axis=1)]
        valid_indices = X_test_clean.index

        if len(valid_indices) == 0:
            continue

        # Elegir modelo según tipo de dato
        if y_train.dtype == 'object' or y_train.dtype == 'bool':
            model = RandomForestClassifier(n_estimators=100, random_state=0)
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=0)

        # Entrenar y predecir
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test_clean)

        # Conversión si es bool o categórica
        if y_train.dtype == 'bool':
            y_pred = y_pred.astype(bool)
        elif y_train.dtype == 'object':
            y_pred = pd.Series(y_pred).astype(df_imputado[col].dtype).values

        # Imputar
        df_imputado.loc[valid_indices, col] = y_pred

    i += 1

print("Imputación completada.")
df = df_imputado

# Los NaN que quedan los cambiamos por su media
df = df.fillna(df.mean(numeric_only=True))

(4277, 13)
Iteración 1
Iteración 2
Iteración 3
Iteración 4
Iteración 5
Imputación completada.


In [9]:
# Preparamos X_test
X_test = df

# Hacemos la prediccion
y_pred = best_model.predict(X_test)

# Se une la prediccion con los passengerIds
submit = pd.DataFrame({
    "PassengerId": index,       # Asegurate de tener esta columna
    "Transported": y_pred
})

submit
submit.to_csv("submission.csv", index=False)
